<a href="https://colab.research.google.com/github/krmonline/AFSCTraining/blob/master/Web_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install pythainlp > /dev/null
!pip install deepcut > /dev/null

In [26]:
import pandas as pd
import re
import collections
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from urllib.parse import quote, unquote
import datetime as dt
import deepcut
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag 
from pythainlp.tag.named_entity import ThaiNameTagger

In [76]:
past_day = 7
keyward = "ตรุษจีน"

In [77]:
now = dt.datetime.now()
past_ts = now.timestamp() - 86400*past_day
past = dt.datetime.fromtimestamp(past_ts)
past_iso = past.isoformat()

In [78]:
header = []
array_news = []
months = {'ม.ค.':'01','ก.พ.':'02','มี.ค.':'03','เม.ย':'04','พ.ค.':'05','มิ.ย.':'06','ก.ค.':'07','ส.ค.':'08','ก.ย.':'09','ต.ค.':'10','พ.ย.':'11','ธ.ค.':'12'}
topics = {'สังคม':'social','การเมือง':'politic','อื่นๆ':'others'}
for j in range(0,20,10):
    #print(j)
    url = "https://search.posttoday.com/search/result?start="+str(j)+"&q="+keyward+"&category=news&sort=relevance&rows=10&xDateSearchRadio=&section=&xChannel=&xNewsSection=&xColumn=&columnistName=&topic=&people=&location=&searchedField=all&publishedDate=" + past_iso
    #print(url)
    r = get(url)
    content = BeautifulSoup(r.text).find_all('div', class_="detail")
    if len(content) < 1:
      break
    #print(content)
    for i in content:
        array_new = {}
        tmp = i.find_all('a')
        text = tmp[1].text
        if tmp[0].text == 'สังคม':
          topic = 'social'
        elif tmp[0].text == 'การเมือง':
            topic = 'politic'
        elif tmp[0].text == 'บันเทิง':
          topic = 'entertainment'
        else:
            topic = "others"
        link = tmp[1].get('href')
        mid = link.split('news')[-1]
        array_new["id"] = mid
        array_new["title"] = text
        if(text == "Online"): #Fix Bug
          continue
        print(text,link)
        #header.append()
        r2 = get(link)
        content = BeautifulSoup(r2.text).find('div', class_="article-content")
        dt_thai = BeautifulSoup(r2.text).find('div', class_="date_time")
        arr_dt = dt_thai.text.split(" ")
        arr_new_dt = [str(int(arr_dt[3])-543),months[arr_dt[2]],arr_dt[1]]
        new_dt = " ".join(["-".join(arr_new_dt),arr_dt[5]+":00"])
        array_new['time'] = new_dt
        array_new['content'] = content.text.strip()
        array_new['tag'] = []
        array_new["link"] = link
        array_new["source"] = "posttoday"
        array_new['topic'] = topic
        #id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
        array_news.append(array_new)
df = pd.DataFrame(array_news)

คิงเพาเวอร์ มองกำลังซื้อระลอกแรก เดือน ก.พ. ผ่าน 2 อีเวนต์ ตรุษจีน-วาเลนไทนส์ https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Feconomy%2Fnews%2F645000&contentId=pt-news645000
สธ.เปิดข้อมูลผู้ติดเชื้อพบมากในตลาดสด เป็นคนขายถึง90.19% https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Fsocial%2Fgeneral%2F644999&contentId=pt-news644999
ก็หมวยนี่คะ! เขื่อน K-otic จัดเต็มด้วยแฟชั่นกี่เพ้าสุดปังรับตรุษจีน https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Fent%2Fnews%2F644976&contentId=pt-news644976
มาแล้ว!คลิปแรกบิ๊กตู่ในTikTok อวยพรตรุษจีน https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Fpolitic%2Fnews%2F644977&contentId=pt-news644977
วันจ่ายตรุษจีนเมืองพัทยาไม่คึกสะท้อนความบอบช้ำโควิด-19ระบาด https://search.posttoday.com/track/visitAndRedirect?href=https%3A%2F%2Fwww.posttoday.com%2Fsocial%2Flocal%2F644964&contentId=pt-news644964
ว

In [84]:
df = pd.DataFrame(array_news)
df

,id,title,time,content,tag,link,source,topic
0,645000,คิงเพาเวอร์ มองกำลังซื้อระลอกแรก เดือน ก.พ. ผ่...,2021-02-10 13:40:00,"กลุ่มคิง เพาเวอร์ ต่อยอดกลยุทธ์ ""ช้อปได้โดยไม่...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,others
1,644999,สธ.เปิดข้อมูลผู้ติดเชื้อพบมากในตลาดสด เป็นคนขา...,2021-02-10 13:32:00,โฆษก ศบค. เผย เล็งเก็บข้อมูลปล่อยให้คนโรงงานสม...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social
2,644976,ก็หมวยนี่คะ! เขื่อน K-otic จัดเต็มด้วยแฟชั่นกี...,2021-02-10 11:21:00,เขื่อน ภัทรดนัย เริ่ดไม่หยุดทุกลุค จัดเต็มแฟชั...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,entertainment
3,644977,มาแล้ว!คลิปแรกบิ๊กตู่ในTikTok อวยพรตรุษจีน,2021-02-10 11:22:00,"ทีมโฆษกรัฐบาลเผยคลิปแรกของ ""พล.อ.ประยุทธ์"" ในT...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic
4,644964,วันจ่ายตรุษจีนเมืองพัทยาไม่คึกสะท้อนความบอบช้ำ...,2021-02-10 09:30:00,ชลบุรี-วันจ่ายตรุษจีนตลาดอมรนาเกลือและตลาดใหม่...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social
5,644962,วันจ่ายรับตรุษจีนปากน้ำโพ พบสินค้าเซ่นไหว้ปรับ...,2021-02-10 08:57:00,นครสวรรค์-ปากน้ำโพคึกรับวันจ่ายตรุษจีนชาวไทยเช...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social
6,644960,"โพลล์ ชี้ ตรุษจีนปีนี้ แยกกันไหว้ ""งดเที่ยว-งด...",2021-02-10 08:44:00,กรุงเทพโพลล์ เผย ประชาชน ร้อยละ 58.2 ระบุ วันไ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social
7,644939,"""ประยุทธ์""เตรียมโผล่ ""TikTok"" ขอทีมงานใส่เพลงเ...",2021-02-09 19:01:00,"นายกฯเตรียมโผล่ใน ""TikTok"" ประเดิมโพสต์คลิปแรก...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic
8,644920,เจ้าอาวาสวัดเล่งเน่ยยี่อวยพร“บิ๊กตู่”มอบวัวกระ...,2021-02-09 17:10:00,“ธนกร”เผย เจ้าอาวาสวัดเล่งเน่ยยี่อวยพรให้ “บิ๊...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic
9,644902,ธอส.ฉลองตรุษจีน ประมูลบ้านมือสองออนไลน์ 999 รา...,2021-02-09 14:41:00,ธอส. มอบอั่งเปาด้วยส่วนลดพิเศษอีก 10% จากราคาท...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others


In [85]:
ner = ThaiNameTagger()

In [87]:
#id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
header2 = []
tags = [] 
arr_person = []
arr_date = []
arr_org = []
arr_location = []
arr_phone = []
arr_url = []
arr_zip = []
for i in df.content.values[:]:
    arr_text=word_tokenize(i, engine="deepcut")
    #p = pos_tag(arr_text,corpus='orchid_ud')
    #print(p)
    tmp = " ".join(arr_text)
    header2.append(tmp)
    tag = ner.get_ner(i,pos=False,tag=True)
    s = BeautifulSoup(tag)
    arr_person_tmp = []
    for i  in s.find_all("person"):
      person_tmp = i.contents[0].split('.')[-1]
      person_tmp = person_tmp.split('นาย')[-1]
      person_tmp = person_tmp.split('นาง')[-1]
      person_tmp = person_tmp.split('นางสาว')[-1]
      arr_person_tmp.append(person_tmp)
    person = ",".join(arr_person_tmp)
    arr_person.append(person)
    date = ",".join([i.contents[0] for i  in s.find_all("date")])
    arr_date.append(date)
    org = ",".join([i.contents[0] for i  in s.find_all("organization")])
    arr_org.append(org)
    location = ",".join([i.contents[0] for i  in s.find_all("location")])
    print(location)
    arr_location.append(location)
    phone = ",".join([i.contents[0] for i  in s.find_all("phone")])
    arr_phone.append(phone)
    url = ",".join([i.contents[0] for i  in s.find_all("url")])
    arr_url.append(url)
    zip = ",".join([i.contents[0] for i  in s.find_all("zip")])
    arr_zip.append(zip)
#df['title_tk'] = header2
df['location'] = arr_location
df['organization'] = arr_org
df['person'] = arr_person
df['phone'] = arr_phone
df['url'] = arr_url
df['zip'] = arr_zip


df

กรุงเทพฯ,ภูเก็ต,มหานคร,ภูเก็ต,ไทย,ไทย,กรุงเทพฯ ใจกลางสาทร,ลานมหานคร สแควร์,ไทย
จังหวัดสมุทรสาคร,จังหวัดสมุทรสาคร,กรุงเทพฯ

ไทยคู่ฟ้า,จีน,จีน,ไทยคู่ฟ้า?,ไทยคู่ฟ้า
ชลบุรี,อ.บางละมุง,จ.ชลบุรี,จีน,เมืองพัทยา,สมุทรสาคร
นครสวรรค์,จีน,เทศบาลนครนครสวรรค์,อ.เมือง,จ.นครสวรรค์,จีน,จังหวัดนครสวรรค์บอ
กรุงเทพโพลล์,จีน
ทำเนียบรัฐบาล
ไทย
บ้านมือสอง,กรุงเทพฯ,ปริมณฑล,อ.พระประแดง,จ.สมุทรปราการ,อ.เมืองปทุมธานี,หมู่บ้านชวนชมรัตน์,อ.บางบัวทอง,จ.นนทบุรี,สถานีคลองบางไผ่,ศูนย์การค้าเซ็นทรัลพลาซ่า เวสต์เกต,คลองหลวง,อ.คลองหลวง,จ.ปทุมธานี,ศูนย์การค้าฟิวเจอร์พาร์ค,รังสิต,จ.จันทบุรี,อ.เมืองสงขลา,จ.สงขลา,อ.เมืองนครพนม,จ.นครพนม
ตลาดรถไฟมหาชัย,สมุทรสาคร,จ.สมุทรสงคราม,อ.อัมพวา,ตลาดรถไฟ,ต.มหาชัย,อ.เมือง,จ.สมุทรสาคร,จ.สมุทรสาคร,ตลาดรถไฟ,ต.มหาชัย"แม่ค้าขาย,อ.อัมพวา,ต.มหาชัย,ไทย,เมียนมา,ลาว,กทม.,ไทย

จีน,จีน,จีน,จีน,จีน,จีน,คอลเลกชั่น Xin,สยามสแควร์วัน, เซ็นทรัลเวิลด์
ไทย,ประเทศไทย,ไทย,เอเชียเช้านี้,ยุโรป,สหรัฐฯ,สหรัฐฯ,ไทย,แนวโน้มหุ้นไทย2ก.พ.64

จีน,จีน,จีน,จีน,จีน,จีน,จีน,ประเทศคู่ค้า,จีน,จีน,จีน,จีน,จีน,จีน

จีน,ไทย,ไทย

,id,title,time,content,tag,link,source,topic,location,organization,person,phone,url,zip
0,645000,คิงเพาเวอร์ มองกำลังซื้อระลอกแรก เดือน ก.พ. ผ่...,2021-02-10 13:40:00,"กลุ่มคิง เพาเวอร์ ต่อยอดกลยุทธ์ ""ช้อปได้โดยไม่...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"กรุงเทพฯ,ภูเก็ต,มหานคร,ภูเก็ต,ไทย,ไทย,กรุงเทพฯ...","กลุ่มคิง,บริษัทมองว่า,บริษัท คิง เพาเวอร์ ผู้ด...",บริษัทคิงเพาเวอร์ ผู้บริหารธุรกิจค้าปลีกศูนย์ก...,,"www.kingpower.com,www.kingpower.com",
1,644999,สธ.เปิดข้อมูลผู้ติดเชื้อพบมากในตลาดสด เป็นคนขา...,2021-02-10 13:32:00,โฆษก ศบค. เผย เล็งเก็บข้อมูลปล่อยให้คนโรงงานสม...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"จังหวัดสมุทรสาคร,จังหวัดสมุทรสาคร,กรุงเทพฯ","สธ.,สาธารณสุขจังหวัดสมุทรสาคร,แอนด์ซีล,กระทรวง...","ทวีศิลป์,วิษณุโยธิน,โรคติดเชื้อไวรัสโคโรนา 2019",,,
2,644976,ก็หมวยนี่คะ! เขื่อน K-otic จัดเต็มด้วยแฟชั่นกี...,2021-02-10 11:21:00,เขื่อน ภัทรดนัย เริ่ดไม่หยุดทุกลุค จัดเต็มแฟชั...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,entertainment,,,"เขื่อน ภัทรดนัย,เขื่อน ภัทรดนัย,เขื่อน เคโอติก...",,,
3,644977,มาแล้ว!คลิปแรกบิ๊กตู่ในTikTok อวยพรตรุษจีน,2021-02-10 11:22:00,"ทีมโฆษกรัฐบาลเผยคลิปแรกของ ""พล.อ.ประยุทธ์"" ในT...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic,"ไทยคู่ฟ้า,จีน,จีน,ไทยคู่ฟ้า?,ไทยคู่ฟ้า","สำนักโฆษก สำนักเลขาธิการนายกรัฐมนตรี,กลาโหม","ประยุทธ์,ประยุทธ์ จันทร์โอชา,แอคเคาท์ thaigov,...",,,
4,644964,วันจ่ายตรุษจีนเมืองพัทยาไม่คึกสะท้อนความบอบช้ำ...,2021-02-10 09:30:00,ชลบุรี-วันจ่ายตรุษจีนตลาดอมรนาเกลือและตลาดใหม่...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"ชลบุรี,อ.บางละมุง,จ.ชลบุรี,จีน,เมืองพัทยา,สมุท...",,,,,
5,644962,วันจ่ายรับตรุษจีนปากน้ำโพ พบสินค้าเซ่นไหว้ปรับ...,2021-02-10 08:57:00,นครสวรรค์-ปากน้ำโพคึกรับวันจ่ายตรุษจีนชาวไทยเช...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"นครสวรรค์,จีน,เทศบาลนครนครสวรรค์,อ.เมือง,จ.นคร...",,สาวประณีต เรืองพล,,,
6,644960,"โพลล์ ชี้ ตรุษจีนปีนี้ แยกกันไหว้ ""งดเที่ยว-งด...",2021-02-10 08:44:00,กรุงเทพโพลล์ เผย ประชาชน ร้อยละ 58.2 ระบุ วันไ...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,social,"กรุงเทพโพลล์,จีน","กรุงเทพโพลล์,ศูนย์วิจัยมหาวิทยาลัยกรุงเทพ,สธ.",,,,
7,644939,"""ประยุทธ์""เตรียมโผล่ ""TikTok"" ขอทีมงานใส่เพลงเ...",2021-02-09 19:01:00,"นายกฯเตรียมโผล่ใน ""TikTok"" ประเดิมโพสต์คลิปแรก...",[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic,ทำเนียบรัฐบาล,"มหาดไทย,สำนักนายกรัฐมนตรี,กลาโหม,สำนักนายกรัฐม...","อนุชา บูรพชัยศรี,ประยุทธ์ จันทร์โอชา,บิ๊กตู่,...",,,
8,644920,เจ้าอาวาสวัดเล่งเน่ยยี่อวยพร“บิ๊กตู่”มอบวัวกระ...,2021-02-09 17:10:00,“ธนกร”เผย เจ้าอาวาสวัดเล่งเน่ยยี่อวยพรให้ “บิ๊...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,politic,ไทย,"สำนักนายกรัฐมนตรี,วัดมังกรกมลาวาส,กระทรวงกลาโห...","บิ๊กตู่,ผ่านศึกซักฟอก พร้อมอวยพรสุขภาพแข็งแรง,...",,,
9,644902,ธอส.ฉลองตรุษจีน ประมูลบ้านมือสองออนไลน์ 999 รา...,2021-02-09 14:41:00,ธอส. มอบอั่งเปาด้วยส่วนลดพิเศษอีก 10% จากราคาท...,[],https://search.posttoday.com/track/visitAndRed...,posttoday,others,"บ้านมือสอง,กรุงเทพฯ,ปริมณฑล,อ.พระประแดง,จ.สมุท...","ธอส.,ธนาคารอาคารสงเคราะห์,ธอส.,ธอส.,มหาวิทยาลั...","ฉัตรชัย ศิริไล,ปทุมธานีส่วนทรัพย์เด่นทำเลดี อา...",,"www.ghbhomecenter.com,www.ghbank.co.th",


In [60]:
#atag = ner.get_ner(text,pos=False,tag=True)
#tag

'<LOCATION>พิจิตร</LOCATION>-สลด! โรงงานพลุระเบิดตาย4เจ็บ3สูญหาย1บ้านพัง5หลัง ผู้ว่าฯรุดตรวจสอบสั่งเร่งตรวจสอบความเสียหายเยียวยาผู้ประสบภัย\t\t\t\t\t\t\t\nเมื่อวันที่ <DATE>4 ก.พ.</DATE> 2564เวลา <TIME>16.45 น.</TIME> ได้เกิดเสียงระเบิดดังสนั่นหวั่นไหวได้ยินไปไกลทั้งหมู่บ้าน จุดเกิดเหตุอยู่ทมี่ หมู่ 3 <LOCATION>ต.หนองหญ้าไทร</LOCATION> <LOCATION>อ.สากเหล็ก</LOCATION> <LOCATION>จ.พิจิตร</LOCATION> เมื่อสิ้นเสียงระเบิดก็พบเห็นเปลวไฟพวยพุ่ง เนื่องจากเกิดไฟไหม้บ้านพร้อมๆกันถึง 5 หลังคาเรือนชาวบ้านที่เห็นเหตุการณ์รีบโทรแจ้งตำรวจและหน่วยกู้ภัยจากนั้นรถดับเพลิงในพื้นที่ข้างเคียงของ <LOCATION>ต.หนองหญ้าไทร</LOCATION> จำนวนนับสิบคันต่างวิ่งเร่งเข้าดับเพลิง จากนั้นตำรวจ และหน่วยกู้ภัยเข้าเคลียร์พื้นที่ก็พบว่า เกิดจากการระเบิดของวัตถุระเบิดชนิดหนึ่ง ซึ่งน่าจะมีเป็นจำนวนมากเบื้องต้นสันนิษฐานว่า บ้าน<PERSON>ต้นเหตุน่าจะเป็นสถานที่เก็บพลุ</PERSON> หรือวัตถุระเบิดอย่างใดอย่างหนึ่งต้องรอตำรวจพิสูจน์หลักฐานตรวจที่เกิดเหตุ อย่างไรก็ตามมีผู้เสียชีวิตจากเหตุระเบิด 4 ราย สภาพศพระบุไม่ได้ว่าเป็นชายหรือหญิง 

In [63]:
#id|title|time|content|tag|link|source|topic|location|organization|person|phone|url|zip|things
#df